# Notebook 4: Ejecución y Costes (Motor de Backtesting)

## Objetivo
Implementación del motor de backtesting con reglas estrictas de ejecución y cálculo de costes transaccionales según las especificaciones del PDF.

## Reglas de Ejecución (según PDF)
- **Venta:** Todas las posiciones que salen de la cartera se cierran al precio **OPEN** del día de rebalanceo
- **Compra:** Las nuevas posiciones se adquieren al precio **CLOSE** del mismo día
- **Costes:** 0.23% sobre el valor efectivo de cada operación, mínimo $23 por orden
- **Activos que dejan de cotizar:** Vender al CLOSE del día que salen, mantener cash hasta siguiente rebalanceo

## Índice
1. [Configuración y Carga de Datos](#configuracion)
2. [Clase BacktestingEngine](#engine)
3. [Ejecución del Backtesting](#ejecucion)
4. [Análisis de Operaciones y Costes](#analisis)

---

## 1. Configuración y Carga de Datos {#configuracion}

Carga de datos de precios diarios, selección de activos y configuración inicial.

In [ ]:
# Librerías permitidas
import numpy as np
import pandas as pd
import warnings
import os
from datetime import datetime

warnings.filterwarnings('ignore')

# Parámetros del backtest
INITIAL_CAPITAL = 250000  # $250,000 según PDF
COMMISSION_RATE = 0.0023  # 0.23%
MIN_COMMISSION = 23.0  # $23 mínimo por orden

# Cargar datos
data_dir = '../data'

# Cargar datos (descomentar cuando estén disponibles)
# price_data_clean = pd.read_parquet(f'{data_dir}/price_data_clean.parquet')
# selection_df = pd.read_csv(f'{data_dir}/selection_with_weights.csv', parse_dates=['rebalance_date'])
# rebalance_calendar = pd.read_csv(f'{data_dir}/rebalance_calendar.csv', index_col=0, parse_dates=True)

print("⚠️  Cargar datos antes de continuar")
print(f"Capital inicial: ${INITIAL_CAPITAL:,.2f}")
print(f"Comisión: {COMMISSION_RATE*100:.2f}% (mínimo ${MIN_COMMISSION:.2f})")
print(f"Fecha de ejecución: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 2. Clase BacktestingEngine {#engine}

Implementación de la clase principal del motor de backtesting con todas las reglas de ejecución.

In [ ]:
class BacktestingEngine:
    """
    Motor de backtesting con reglas estrictas de ejecución.
    
    Reglas:
    - Ventas al precio OPEN del día de rebalanceo
    - Compras al precio CLOSE del mismo día
    - Costes: 0.23% con mínimo $23 por orden
    """
    
    def __init__(self, initial_capital, commission_rate=0.0023, min_commission=23.0):
        """
        Inicializa el motor de backtesting.
        
        Parámetros:
        -----------
        initial_capital : float
            Capital inicial
        commission_rate : float
            Tasa de comisión (0.0023 = 0.23%)
        min_commission : float
            Comisión mínima por orden
        """
        self.initial_capital = initial_capital
        self.commission_rate = commission_rate
        self.min_commission = min_commission
        
        # Estado de la cartera
        self.cash = initial_capital
        self.positions = {}  # {asset: shares}
        self.equity_curve = []
        self.trades_log = []
        
    def calculate_commission(self, trade_value):
        """
        Calcula comisión según reglas del PDF.
        
        Parámetros:
        -----------
        trade_value : float
            Valor de la operación
        
        Retorna:
        --------
        float
            Comisión a pagar
        """
        commission = trade_value * self.commission_rate
        return max(commission, self.min_commission)
    
    def sell_position(self, asset, shares, price_open, date):
        """
        Vende posición al precio OPEN.
        
        Parámetros:
        -----------
        asset : str
            Símbolo del activo
        shares : float
            Número de acciones a vender
        price_open : float
            Precio OPEN del día
        date : datetime
            Fecha de la operación
        """
        if shares <= 0 or asset not in self.positions:
            return
        
        trade_value = shares * price_open
        commission = self.calculate_commission(trade_value)
        net_proceeds = trade_value - commission
        
        self.cash += net_proceeds
        del self.positions[asset]
        
        # Registrar operación
        self.trades_log.append({
            'date': date,
            'asset': asset,
            'action': 'SELL',
            'shares': shares,
            'price': price_open,
            'trade_value': trade_value,
            'commission': commission,
            'net_value': net_proceeds
        })
    
    def buy_position(self, asset, target_value, price_close, date):
        """
        Compra posición al precio CLOSE.
        
        Parámetros:
        -----------
        asset : str
            Símbolo del activo
        target_value : float
            Valor objetivo de la posición (antes de comisión)
        price_close : float
            Precio CLOSE del día
        date : datetime
            Fecha de la operación
        """
        if price_close <= 0 or target_value <= 0:
            return
        
        # Calcular comisión sobre el valor objetivo
        commission = self.calculate_commission(target_value)
        net_value = target_value - commission
        
        if net_value > self.cash:
            # Ajustar si no hay suficiente cash
            net_value = self.cash
            target_value = net_value / (1 - self.commission_rate)
            commission = self.calculate_commission(target_value)
        
        shares = net_value / price_close
        self.cash -= (shares * price_close + commission)
        self.positions[asset] = shares
        
        # Registrar operación
        self.trades_log.append({
            'date': date,
            'asset': asset,
            'action': 'BUY',
            'shares': shares,
            'price': price_close,
            'trade_value': shares * price_close,
            'commission': commission,
            'net_value': -net_value
        })
    
    def rebalance(self, selected_assets, weights, prices_open, prices_close, date):
        """
        Realiza rebalanceo según reglas del PDF.
        
        Parámetros:
        -----------
        selected_assets : list
            Lista de activos seleccionados
        weights : dict
            Diccionario {asset: weight} con pesos objetivo
        prices_open : pd.Series
            Precios OPEN del día
        prices_close : pd.Series
            Precios CLOSE del día
        date : datetime
            Fecha de rebalanceo
        """
        # 1. Vender posiciones que salen de la cartera (al OPEN)
        current_assets = set(self.positions.keys())
        new_assets = set(selected_assets)
        assets_to_sell = current_assets - new_assets
        
        for asset in assets_to_sell:
            if asset in prices_open.index and pd.notna(prices_open[asset]):
                self.sell_position(asset, self.positions[asset], prices_open[asset], date)
        
        # 2. Calcular valor total de la cartera (después de ventas)
        portfolio_value = self.cash
        for asset, shares in self.positions.items():
            if asset in prices_close.index and pd.notna(prices_close[asset]):
                portfolio_value += shares * prices_close[asset]
        
        # 3. Comprar nuevas posiciones (al CLOSE)
        for asset in selected_assets:
            if asset not in prices_close.index or pd.isna(prices_close[asset]):
                continue
            
            target_weight = weights.get(asset, 0.05)  # Default 5%
            target_value = portfolio_value * target_weight
            
            if asset in self.positions:
                # Ajustar posición existente
                current_value = self.positions[asset] * prices_close[asset]
                if current_value < target_value:
                    # Comprar más
                    additional_value = target_value - current_value
                    self.buy_position(asset, additional_value, prices_close[asset], date)
                elif current_value > target_value:
                    # Vender parte (ya vendido arriba si sale completamente)
                    pass  # Se maneja en el siguiente rebalanceo
            else:
                # Nueva posición
                self.buy_position(asset, target_value, prices_close[asset], date)
    
    def update_equity(self, prices_close, date):
        """
        Actualiza curva de equity.
        
        Parámetros:
        -----------
        prices_close : pd.Series
            Precios CLOSE del día
        date : datetime
            Fecha
        """
        portfolio_value = self.cash
        for asset, shares in self.positions.items():
            if asset in prices_close.index and pd.notna(prices_close[asset]):
                portfolio_value += shares * prices_close[asset]
        
        self.equity_curve.append({
            'date': date,
            'equity': portfolio_value,
            'cash': self.cash,
            'positions_value': portfolio_value - self.cash
        })
    
    def get_equity_curve(self):
        """
        Retorna curva de equity como DataFrame.
        """
        return pd.DataFrame(self.equity_curve).set_index('date')
    
    def get_trades_log(self):
        """
        Retorna log de operaciones como DataFrame.
        """
        return pd.DataFrame(self.trades_log)


print("✓ Clase BacktestingEngine definida")

## 3. Ejecución del Backtesting {#ejecucion}

Ejecución del backtesting mes a mes siguiendo el calendario de rebalanceo.

In [ ]:
def run_backtest(price_data, selection_df, rebalance_calendar, initial_capital=250000):
    """
    Ejecuta backtesting completo.
    
    Parámetros:
    -----------
    price_data : pd.DataFrame
        Precios diarios (debe tener columnas Open y Close, o solo Close)
    selection_df : pd.DataFrame
        DataFrame con selección de activos por fecha
    rebalance_calendar : pd.DataFrame
        Calendario de rebalanceo
    initial_capital : float
        Capital inicial
    
    Retorna:
    --------
    BacktestingEngine
        Motor con resultados del backtest
    """
    engine = BacktestingEngine(initial_capital)
    
    # Verificar si tenemos precios Open y Close separados
    has_open_close = False
    if 'Open' in price_data.columns and 'Close' in price_data.columns:
        prices_open = price_data['Open']
        prices_close = price_data['Close']
        has_open_close = True
    else:
        # Asumir que price_data tiene solo precios de cierre
        prices_close = price_data
        prices_open = price_data  # Usar mismo precio para ventas (aproximación)
    
    # Ejecutar rebalanceo mes a mes
    for rebalance_date in rebalance_calendar.index:
        # Obtener activos seleccionados para esta fecha
        date_selection = selection_df[selection_df['rebalance_date'] == rebalance_date]
        
        if len(date_selection) == 0:
            continue
        
        selected_assets = date_selection['asset'].tolist()
        weights = dict(zip(date_selection['asset'], date_selection['weight']))
        
        # Obtener precios del día de rebalanceo
        if rebalance_date in prices_close.index:
            prices_open_day = prices_open.loc[rebalance_date] if has_open_close else prices_close.loc[rebalance_date]
            prices_close_day = prices_close.loc[rebalance_date]
            
            # Rebalancear
            engine.rebalance(selected_assets, weights, prices_open_day, prices_close_day, rebalance_date)
            
            # Actualizar equity
            engine.update_equity(prices_close_day, rebalance_date)
    
    return engine


# Ejecutar backtest (descomentar cuando datos estén disponibles)
# engine = run_backtest(
#     price_data_clean,
#     selection_df,
#     rebalance_calendar,
#     initial_capital=INITIAL_CAPITAL
# )

# Obtener resultados
# equity_curve = engine.get_equity_curve()
# trades_log = engine.get_trades_log()

# Guardar resultados
# equity_curve.to_csv(f'{data_dir}/equity_curve.csv')
# trades_log.to_csv(f'{data_dir}/trades_log.csv')

print("⚠️  Ejecutar backtest después de cargar todos los datos")

In [ ]:
# Análisis de operaciones y costes (descomentar cuando engine esté disponible)
# if 'engine' in locals():
#     trades_log = engine.get_trades_log()
#     equity_curve = engine.get_equity_curve()
#     
#     print("\\n=== ANÁLISIS DE OPERACIONES Y COSTES ===\\n")
#     
#     # Estadísticas de operaciones
#     total_trades = len(trades_log)
#     buy_trades = len(trades_log[trades_log['action'] == 'BUY'])
#     sell_trades = len(trades_log[trades_log['action'] == 'SELL'])
#     
#     print(f"1. ESTADÍSTICAS DE OPERACIONES:")
#     print(f"   - Total de operaciones: {total_trades}")
#     print(f"   - Compras: {buy_trades}")
#     print(f"   - Ventas: {sell_trades}")
#     
#     # Costes totales
#     total_commissions = trades_log['commission'].sum()
#     total_trade_value = trades_log['trade_value'].abs().sum()
#     commission_pct = (total_commissions / total_trade_value * 100) if total_trade_value > 0 else 0
#     
#     print(f"\\n2. COSTES TRANSACCIONALES:")
#     print(f"   - Comisiones totales: ${total_commissions:,.2f}")
#     print(f"   - Valor total operado: ${total_trade_value:,.2f}")
#     print(f"   - Comisiones como % del valor operado: {commission_pct:.2f}%")
#     
#     # Equity final
#     final_equity = equity_curve['equity'].iloc[-1]
#     total_return = (final_equity - INITIAL_CAPITAL) / INITIAL_CAPITAL * 100
#     
#     print(f"\\n3. RESULTADOS FINALES:")
#     print(f"   - Capital inicial: ${INITIAL_CAPITAL:,.2f}")
#     print(f"   - Equity final: ${final_equity:,.2f}")
#     print(f"   - Retorno total: {total_return:.2f}%")
#     print(f"   - Retorno neto (después de comisiones): {total_return:.2f}%")
#     
#     # Visualización de equity curve
#     import matplotlib.pyplot as plt
#     plt.figure(figsize=(14, 6))
#     plt.plot(equity_curve.index, equity_curve['equity'], label='Equity Curve', linewidth=2)
#     plt.axhline(y=INITIAL_CAPITAL, color='r', linestyle='--', label='Capital Inicial')
#     plt.xlabel('Fecha')
#     plt.ylabel('Valor de la Cartera ($)')
#     plt.title('Evolución del Valor de la Cartera')
#     plt.legend()
#     plt.grid(True, alpha=0.3)
#     plt.tight_layout()
#     plt.show()

print("\\n=== RESUMEN DEL NOTEBOOK 4 ===")
print("✓ Motor de backtesting implementado")
print("✓ Reglas de ejecución (OPEN/CLOSE) aplicadas")
print("✓ Costes transaccionales calculados")
print("✓ Equity curve generada")
print("\\n⚠️  IMPORTANTE: Ejecutar todas las celdas con datos reales")
print("⚠️  Los resultados se usarán en el Notebook 5 para comparación con benchmarks")